# Standards-Compliant Search Logging Demo

This notebook demonstrates how to use the PyEuropePMC search logging utilities to create a PRISMA/Cochrane-compliant, auditable search log for systematic reviews.

Features shown:
- Structured logging of search queries and filters
- Optional persistence of raw search results
- Cryptographic signing and archiving of logs/results
- PRISMA-style summary output

In [8]:
# Import the search logging utilities
from pyeuropepmc.utils.search_logging import (
    start_search, record_query, record_results, prisma_summary,
    sign_and_zip_results
)

## 1. Start a New Search Log
Create a new log for your systematic review search.

In [9]:
log = start_search(title="Immunotherapy in Cancer SR", executed_by="Alice Smith")
log

SearchLog(title='Immunotherapy in Cancer SR', executed_by='Alice Smith', created_at='2025-10-22T15:54:45.826632', entries=[], deduplicated_total=None, final_included=None)

## 2. Record a Search Query
Log the exact search string, filters, and (optionally) persist raw results.

In [3]:
# Perform a real search using SearchClient and log the query/results
from pyeuropepmc.search import SearchClient
real_query = "immunotherapy AND cancer AND PUB_YEAR:[2020 TO 2025]"
filters = {"open_access": True, "language": "English"}
with SearchClient() as client:
    real_results = client.search(real_query, pageSize=5)
    hit_count = real_results.get('hitCount', 0)
    print(hit_count)
    record_query(
    	log,
    	database="Europe PMC",
    	query=real_query,
    	filters=filters,
    	results_returned=hit_count,
    	notes="Real API search",
    	raw_results=real_results,
    	raw_results_dir="./demo_raw_results"
    )

[pyeuropepmc] INFO: SearchClient initialized with cache disabled
[pyeuropepmc] INFO: Cache miss - performing search with params: {'query': 'immunotherapy AND cancer AND PUB_YEAR:[2020 TO 2025]', 'resultType': 'lite', 'synonym': 'FALSE', 'pageSize': 5, 'format': 'json', 'cursorMark': '*', 'sort': ''}
[pyeuropepmc] INFO: GET request to https://www.ebi.ac.uk/europepmc/webservices/rest/search with params={'query': 'immunotherapy AND cancer AND PUB_YEAR:[2020 TO 2025]', 'resultType': 'lite', 'synonym': 'FALSE', 'pageSize': 5, 'format': 'json', 'cursorMark': '*', 'sort': ''} and stream=False
[pyeuropepmc] INFO: Cache miss - performing search with params: {'query': 'immunotherapy AND cancer AND PUB_YEAR:[2020 TO 2025]', 'resultType': 'lite', 'synonym': 'FALSE', 'pageSize': 5, 'format': 'json', 'cursorMark': '*', 'sort': ''}
[pyeuropepmc] INFO: GET request to https://www.ebi.ac.uk/europepmc/webservices/rest/search with params={'query': 'immunotherapy AND cancer AND PUB_YEAR:[2020 TO 2025]', 'r

228862


## 3. Record Aggregate Counts for PRISMA Reporting
After deduplication and screening, record the final counts.

In [4]:
record_results(log, deduplicated_total=2, final_included=1)

## 4. Save the Search Log
Persist the log to disk as a JSON file.

In [5]:
log_path = log.save("./demo_raw_results/demo_search_log.json")
print(f'Log saved to: {log_path}')

[pyeuropepmc] INFO: Search log saved to demo_raw_results/demo_search_log.json


Log saved to: demo_raw_results/demo_search_log.json


## 5. Generate a PRISMA-Style Summary
Produce a summary for reporting in your methods or PRISMA flow diagram.

In [6]:
summary = prisma_summary(log)
import json
print(json.dumps(summary, indent=2))

{
  "title": "Immunotherapy in Cancer SR",
  "executed_by": "Alice Smith",
  "created_at": "2025-10-22T15:54:44.504625",
  "records_by_database": {
    "Europe PMC": 228862
  },
  "total_records_identified": 228862,
  "deduplicated_total": 2,
  "final_included": 1
}


## 6. (Optional) Sign and Archive the Log and Results
Digitally sign and zip the log and raw results for provenance. (Requires an RSA private key.)

In [7]:
# sign_and_zip_results can be used if you have a private key available.
# Example (uncomment and set your key path):
# sign_and_zip_results(
#     files=["./demo_search_log.json", "./demo_raw_results/Europe_PMC_results_20251022T000000.json"],
#     zip_path="./demo_search_log_archive.zip",
#     private_key_path="/path/to/your/private_key.pem"
# )